In [1]:
import pandas as pd 
import numpy as np
import sqlite3

In [2]:
orders = pd.read_csv("orders.csv")
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [3]:
users = pd.read_json("users.json")
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [4]:
conn = sqlite3.connect(":memory:")
with open("restaurants.sql", "r") as f:
    conn.executescript(f.read())

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

In [5]:
final_df = orders.merge(users, on="user_id", how="left") \
                 .merge(restaurants, on="restaurant_id", how="left")

In [31]:
final_df[final_df["membership"] == "Gold"] \
    .groupby("city")["total_amount"] \
    .sum() \
    .sort_values(ascending=False) \
    .head(1)

city
Chennai    1080909.79
Name: total_amount, dtype: float64

In [32]:
final_df.groupby("cuisine")["total_amount"] \
        .mean() \
        .sort_values(ascending=False) \
        .head(1)

cuisine
Mexican    808.021344
Name: total_amount, dtype: float64

In [8]:
final_df.groupby("user_id")["total_amount"] \
        .sum() \
        .loc[lambda x: x > 1000] \
        .count()

np.int64(2544)

In [9]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0-3.5", "3.6-4.0", "4.1-4.5", "4.6-5.0"]

final_df["rating_range"] = pd.cut(final_df["rating"], bins=bins, labels=labels)

final_df.groupby("rating_range")["total_amount"] \
        .sum() \
        .sort_values(ascending=False)

C:\Users\Dell\AppData\Local\Temp\ipykernel_25076\633639198.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final_df.groupby("rating_range")["total_amount"] \


rating_range
4.6-5.0    2197030.75
4.1-4.5    1960326.26
3.0-3.5    1881754.57
3.6-4.0    1717494.41
Name: total_amount, dtype: float64

In [10]:
final_df[final_df["membership"] == "Gold"] \
    .groupby("city")["total_amount"] \
    .mean() \
    .sort_values(ascending=False)

city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [11]:
restaurant_counts = restaurants.groupby("cuisine")["restaurant_id"].nunique()
revenue = final_df.groupby("cuisine")["total_amount"].sum()

pd.concat([restaurant_counts, revenue], axis=1) \
  .rename(columns={"restaurant_id": "restaurant_count"}) \
  .sort_values("restaurant_count")

,restaurant_count,total_amount
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [12]:
gold_orders = final_df[final_df["membership"] == "Gold"].shape[0]
total_orders = final_df.shape[0]

round((gold_orders / total_orders) * 100)

50

In [27]:
final_df.columns

Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating', 'rating_range', 'quarter'],
      dtype='object')

In [28]:
stats = final_df.groupby("restaurant_name_y") \
                .agg(
                    order_count=("order_id", "count"),
                    avg_order_value=("total_amount", "mean")
                )

stats[stats["order_count"] < 20] \
     .sort_values("avg_order_value", ascending=False)

,order_count,avg_order_value
restaurant_name_y,,
Restaurant_294,13,1040.222308
Restaurant_262,18,1029.473333
Restaurant_77,12,1029.180833
Restaurant_193,15,1026.306667
Restaurant_7,16,1002.140625
...,...,...
Restaurant_184,19,621.828947
Restaurant_498,18,596.815556
Restaurant_192,14,589.972857


In [16]:
final_df.groupby(["membership", "cuisine"])["total_amount"] \
        .sum() \
        .sort_values(ascending=False)

membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [17]:
final_df["order_date"] = pd.to_datetime(final_df["order_date"])
final_df["quarter"] = final_df["order_date"].dt.to_period("Q")

final_df.groupby("quarter")["total_amount"] \
        .sum() \
        .sort_values(ascending=False)

C:\Users\Dell\AppData\Local\Temp\ipykernel_25076\3906472878.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df["order_date"] = pd.to_datetime(final_df["order_date"])


quarter
2023Q3    2037385.10
2023Q4    2018263.66
2023Q1    1993425.14
2023Q2    1945348.72
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64

In [18]:
final_df[final_df["membership"] == "Gold"].shape[0]

4987

In [19]:
final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum()

np.float64(1889366.58)

In [20]:
final_df["user_id"].nunique()

2883

In [21]:
round(
    final_df[final_df["membership"] == "Gold"]["total_amount"].mean(),
    2
)

np.float64(797.15)

In [22]:
final_df[final_df["rating"] >= 4.5].shape[0]

3374

In [30]:
top_city = final_df[final_df["membership"] == "Gold"] \
            .groupby("city")["total_amount"] \
            .sum() \
            .idxmax()

final_df[(final_df["membership"] == "Gold") &
         (final_df["city"] == top_city)].shape[0]

1337